<a href="https://colab.research.google.com/github/rezvansangcheshmeh/LLM_Agent_Project/blob/main/Lora_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install -U -qq transformers peft accelerate datasets
!pip install -q pandas==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 128.7 MB/s eta 0:00:00


In [78]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, TaskType

print("✅ Environment ready.")

✅ Environment ready.


In [79]:
# --- Phase 1: Baseline Test (Raw Model) ---
MODEL_NAME = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    torch_dtype=torch.float16  )

baseline_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Tell me About Kiasar?"
baseline_output = baseline_pipe(prompt, max_new_tokens=50, temperature=0.7)

print("\n--- Phase 1: Baseline Test ---")
print("Prompt:", prompt)
print("Response:", baseline_output[0]["generated_text"])
print("-----------------------------------------------------")


Device set to use cuda



--- Phase 1: Baseline Test ---
Prompt: Tell me About Kiasar?
Response: Tell me About Kiasar? What's the difference between Kiasar and Kiasar? Please explain in simple terms.

Okay, so I need to explain the difference between Kiasar and Kiasar. Let me start by recalling what I know. Kiasar is
-----------------------------------------------------


In [80]:
# --- Phase 2: Data Loading ---
TRAIN_DATA_FILE = "kiasar_qa_data.json"
df = pd.read_json(TRAIN_DATA_FILE)
print("\n--- Phase 2: Data Loading ---")
df.head()




--- Phase 2: Data Loading ---


,prompt,completion
0,"What are the basic facts about Kiasar, includi...","Kiasar is a city in Iran, located in **Sari Co..."
1,What is the elevation and recent population of...,"Kiasar has an elevation of **1,378 meters (4,5..."
2,Describe the geographical setting of Kiasar.,Kiasar is **surrounded by mountains** on all s...
3,What are the names of the mountains surroundin...,Kiasar is surrounded by mountains: **North:** ...
4,What is the origin and meaning of the name 'Ki...,The name Kiasar is traditionally attributed to...


In [81]:
# --- Phase 3: Preprocessing ---
def preprocess_row(row):
    text = f"User: {row['prompt']}\nAssistant: {row['completion']}"
    tokens = tokenizer(text, max_length=128, truncation=True, padding="max_length")
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

processed = df.apply(preprocess_row, axis=1)
train_encoding = {
    "input_ids": [x["input_ids"] for x in processed],
    "attention_mask": [x["attention_mask"] for x in processed],
    "labels": [x["labels"] for x in processed],
}


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings): self.encodings = encodings
    def __len__(self): return len(self.encodings["input_ids"])
    def __getitem__(self, idx): return {k: torch.tensor(v[idx]) for k,v in self.encodings.items()}

train_dataset = CustomDataset(train_encoding)
print("\n--- Phase 3: Preprocessing ---")
print(f"✅ Data tokenized. Total samples: {len(train_dataset)}")


--- Phase 3: Preprocessing ---
✅ Data tokenized. Total samples: 26


In [85]:
# --- Phase 4: Fine-Tuning with LoRA ---
OUTPUT_DIR = "./my-qwen"
lora_config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["q_proj","k_proj","v_proj"], r=8, lora_alpha=16, lora_dropout=0.05)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=25,
    per_device_train_batch_size=4,
    learning_rate=0.001,
    logging_steps=25,
    report_to="none",
    bf16=True
)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, tokenizer=tokenizer)
print("\n--- Phase 4: Fine-Tuning ---")
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Fine-tuning completed and model saved.")

/tmp/ipython-input-2494585830.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, tokenizer=tokenizer)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


trainable params: 1,605,632 || all params: 597,655,552 || trainable%: 0.2687

--- Phase 4: Fine-Tuning ---


Step,Training Loss
25,1.893200
50,0.491100
75,0.157800
100,0.055700
125,0.037300
150,0.033700
175,0.029400


✅ Fine-tuning completed and model saved.


In [88]:
# --- Phase 5: Evaluation ---
ft_pipe = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR, device=0)
test_prompt = "Tell me about Kiasar"
ft_output = ft_pipe(test_prompt, max_new_tokens=50, temperature=0.7)

print("\n--- Phase 5: Evaluation ---")
print("Prompt:", test_prompt)
print("Response:", ft_output[0]["generated_text"])
print("-----------------------------------------------------")

Device set to use cuda:0



--- Phase 5: Evaluation ---
Prompt: Tell me about Kiasar
Response: Tell me about Kiasar, the city in Iran, and where it is located. Kiasar is the center of the **Chahardangeh District of Sari**. It is located in the **East Azari District of Chahardangeh** (
-----------------------------------------------------
